# Lab 1: Getting started with geospatial data analysis

This notebook is part of the blog post [Getting Started with Geospatial Analysis on SageMaker Studio Lab](https://towardsdatascience.com/getting-started-with-geospatial-analysis-b2116c50308b) and covers the basics of getting started with Geospatial Data Analysis on SageMaker Studio Labs. We start with exploring publicly available geographic datasets and then explore the sentinel geospatial dataset available at AWS open data registry. We explore California Lakes and Counties using geographic vector data and then focus Lake Shasta in California for analyzing Sentinel-2 geospatial data and calculating spectral indices. 

<div class="alert alert-info">
This notebook is based on the GitHub repo <code>https://github.com/samx18/geospatial_analysis</code> and is updated for SageMaker AI Studio
</div>


In [ ]:
# install packages
# !python3 -m pip install --force-reinstall --no-cache -q -r requirements.txt

In [ ]:
import psutil
import platform

# CPU Information
print("=== CPU Information ===")
print(f"Physical cores: {psutil.cpu_count(logical=False)}")
print(f"Total cores: {psutil.cpu_count(logical=True)}")
print(f"CPU Frequency: {psutil.cpu_freq().current:.2f}MHz")
print(f"CPU Usage Per Core:")
for i, percentage in enumerate(psutil.cpu_percent(percpu=True, interval=1)):
    print(f"Core {i}: {percentage}%")

# Memory Information
print("\n=== Memory Information ===")
memory = psutil.virtual_memory()
print(f"Total: {memory.total / (1024**3):.2f} GB")
print(f"Available: {memory.available / (1024**3):.2f} GB")
print(f"Used: {memory.used / (1024**3):.2f} GB")
print(f"Percentage: {memory.percent}%")

# Try to get GPU information if available
try:
    print("\n=== GPU Information ===")
    !nvidia-smi
except:
    print("No GPU information available or nvidia-smi not installed")


## Install Packages
Install required packages

In [ ]:
%pip install pandas
%pip install numpy
%pip install geopandas
%pip install shapely
%pip install matplotlib
%pip install plotly_express
%pip install sentinelhub[AWS]
%pip install rasterio
%pip install earthpy
%pip install folium

## Import Packages
After the environment is created and selected or the packages installed manually, we can import them directly.

In [ ]:
import pandas as pd
import numpy as np
import geopandas as gpd
from shapely.geometry import Point
import matplotlib
import matplotlib.pyplot as plt 
import folium
import plotly_express as px
import os
import warnings
warnings.filterwarnings('ignore')

## Download Data
Download shapely files that include the geographic data for California counties and water bodies. 
- The CA Counties dataset contains boundaries for CA State, counties and places from the US Census Bureau's 2016 MAF/TIGER database available at https://data.ca.gov/dataset/ca-geographic-boundaries . 
- The California water bodies dataset is published by California. Department of Fish and Game. Marine Resources Region and is available for download here - https://maps.princeton.edu/download/file/stanford-zx543xm6802-shapefile.zip 

After the files are downloaded, we need to unzip these in your local directories.


In [ ]:
# Download and extract California counties dataset

ca_base_url = 'https://data.ca.gov/dataset/e212e397-1277-4df3-8c22-40721b095f33/resource/b0007416-a325-4777-9295-368ea6b710e6/download/'
ca_data_file = 'ca_counties.zip'

if not os.path.isfile(ca_data_file):
    !wget {ca_base_url}{ca_data_file} --no-check-certificate
    !unzip -o {ca_data_file} -d ca_counties

In [ ]:
# Download and extract California lakes dataset

lakes_base_url = 'https://maps.princeton.edu/download/file/'
lakes_data_file = 'stanford-zx543xm6802-shapefile.zip'
if not os.path.isfile(lakes_data_file):
    !wget {lakes_base_url}{lakes_data_file} --no-check-certificate
    !unzip -o {lakes_data_file} -d ca_lakes

## Geographic EDA
Once downloaded, we can read the data and perform exploratory data analysis. We use the `geopanads` python package that makes it easy to read these shapefiles and create a `geopandas.DataFrame`.

In [ ]:
file = 'ca_counties/CA_Counties.shp' #'CA_Counties/CA_Counties_TIGER2016.shp'
counties = gpd.read_file(file)
counties.shape

In [ ]:
# data check
counties.head()

Once you have the data in a `geopandas.DataFrame` you can easily visualize it. Like standard `pandas.DataFrames`, a `geopandas.DataFrame` has a handy plot method that you can use to create geographic visualizations.

In [ ]:
fig, ax = plt.subplots(figsize=(12,12))
counties.plot(ax=ax,color='xkcd:umber')
plt.title("California counties")
plt.tight_layout()
plt.show()

Similarly we will read the California lakes shapefile into a `geopandas.DataFrame` and visualize it

In [ ]:
file = 'ca_lakes/zx543xm6802.shp'
lakes = gpd.read_file(file)
lakes.shape #check shape

In [ ]:
# data profile
lakes.head()

In [ ]:
fig, ax = plt.subplots(figsize=(12,12))
lakes.plot(ax=ax, color='xkcd:sea blue')
plt.title("California Lakes")
plt.tight_layout()
plt.show()

## Data Wrangling
Let's say we want to overlay the lakes data with the counties data and visualize the lakes along with California counties. Before we can do that,  we need to check and ensure they are projected using the same coordinate reference system (CRS). The `crs` attribute of a `geopanads.DataFrame` does exactly this.  

In [ ]:
counties.crs

In [ ]:
lakes.crs

In our case the California counties and lakes data have different CRS.  So we will re-project the lakes to have the same CRS as counties.

In [ ]:
# re-project lakes to epsg:3857
lakes_projected = lakes.to_crs({'init': 'epsg:3857'})

Once we have both the geographic datasets in the same CRS projection, we can overlay and visualize them. 

In [ ]:
# visualize overlay
fig, ax = plt.subplots(figsize=(14,14))
counties.plot(ax=ax,color='xkcd:umber')
lakes_projected.plot(markersize=1, color='xkcd:sea blue',ax=ax)
plt.title("California Counties and Lakes")
plt.show()

#### Subset Selection
We can select a subset of data from the our `geopandas.DataFrame` to create a new `geopandas.DataFrame` for further analysis and visualization. For our example, let's focus on Lake Shasta. 

In [ ]:
shasta = lakes_projected[lakes_projected['name'] == "Lake Shasta"]
shasta.head()

#### Plotting Selection
Once we have our area of interest selected, it becomes easy to visualize and study it better.

In [ ]:
shasta.plot(color='xkcd:sea blue');

## Working With Geospatial Images
For Geospatial data, we will use Sentinel-2. The [Sentinel-2 mission](https://sentinel.esa.int/web/sentinel/missions/sentinel-2) is a land monitoring constellation of two satellites that provide high resolution optical imagery and continuity for the current SPOT and Landsat missions. The Sentinel-2 dataset is available publicly at the [AWS open data registry](https://registry.opendata.aws/sentinel-2/).

We will use the `sentinelhub` python package, that makes it easy to search and download data specific to our focus area directly from AWS. 

In [ ]:
from sentinelhub import SHConfig
config = SHConfig()

In [ ]:
config

#### Sentinel Hub Setup
This section shows how to configure your credentials for sentinelhub. You can use an optional json file to store and retrieve credentials.

In [ ]:
# %%writefile config.json
# {
#     'sentinelhub':{'instance_id':1}
# }

In [ ]:
# Set your SentinelHub instance id here
sentinelhub_instance_id = 'ad7c1a32-4dcb-4d2a-ab6c-8418d7401719'

In [ ]:
# import json

# with open("config.json") as json_data_file:
#     cfg = json.load(json_data_file)

In [ ]:
import boto3
def get_session_credentials():
    try:
        # Get the current session
        session = boto3.Session()
        
        # Get credentials from the session
        credentials = session.get_credentials()
        
        # Get the frozen credentials (temporary credentials)
        frozen_credentials = credentials.get_frozen_credentials()
        
        # Access the individual credential components
        access_key = frozen_credentials.access_key
        secret_key = frozen_credentials.secret_key
        session_token = frozen_credentials.token
        
        return {
            'aws_access_key_id': access_key,
            'aws_secret_access_key': secret_key,
            'aws_session_token': session_token
        }
    except Exception as e:
        print(f"Error retrieving credentials: {str(e)}")
        return None

In [ ]:
# Get the credentials
credentials = get_session_credentials()

if credentials:
    print("Successfully retrieved temporary credentials")
    # Use credentials as needed
    # Note: It's best practice not to print the actual credentials
else:
    print("Failed to retrieve credentials")

In [ ]:
# test credentials
if credentials:
    s3_client = boto3.client(
        's3',
        aws_access_key_id=credentials['aws_access_key_id'],
        aws_secret_access_key=credentials['aws_secret_access_key'],
        aws_session_token=credentials['aws_session_token']
    )

In [ ]:
# s3_client.list_objects_v2(Bucket='cf-templates-1hiy7ofivmxze-us-west-2')

In [ ]:
# instance_id - Instance ID from from your Sentinel Hub account 
# aws_access_key_id - Access key ID from your AWS account
# aws_secret_access_key - Secrect access key from your AWS account

config.instance_id = sentinelhub_instance_id #cfg["sentinelhub"]["instance_id"]
config.aws_access_key_id = credentials["aws_access_key_id"]
config.aws_secret_access_key = credentials["aws_secret_access_key"]
config.aws_session_token = credentials['aws_session_token']

In [ ]:
# Save the configuration
config.save()

In [ ]:
# Verify credentials

from sentinelhub import WebFeatureService, BBox, CRS, DataCollection, SHConfig
if config.instance_id == '':
    print("Warning! To use WFS functionality, please configure the `instance_id`.")

#### Data Search
Before we download, we need to specify our search coordinates that we want to study and the time window. In our case we are focusing on the Lake Shasta region, which we specify as a bounding box and a random time period.

In [ ]:
# Specify bounding box and time interval for search

search_bbox = BBox(bbox=[-123.050516,37.845040,-122.523172,38.249508], crs=CRS.WGS84)

search_time_interval = ('2019-08-01T00:00:00', '2019-08-15T23:59:59')


wfs_iterator = WebFeatureService(
    search_bbox,
    search_time_interval,
    data_collection=DataCollection.SENTINEL2_L1C,
    maxcc=1.0,
    config=config
)

for tile_info in wfs_iterator:
    print(tile_info)

In [ ]:
# List available tiles
wfs_iterator.get_tiles()

#### Picking Tiles
For best results, we pick a tile with least cloud coverage.

In [ ]:
from sentinelhub.aws import AwsTile

tile_id = 'S2A_OPER_MSI_L1C_TL_VGS2_20200815T224802_A026894_T10TEL_N02.09'
tile_name, time, aws_index = AwsTile.tile_id_to_tile(tile_id)
tile_name, time, aws_index

#### Sentinel Data Download
The Sentinel-2 satellites each carry a single multi-spectral instrument (MSI) with 13 spectral channels in the visible/near infrared (VNIR) and short wave infrared spectral range (SWIR). You can read more about these bands [here](https://en.wikipedia.org/wiki/Sentinel-2#Spectral_bands). For our example will download eight specific bands that will aid our analysis.

In [ ]:
warnings.simplefilter("ignore", UserWarning)
from sentinelhub.aws import AwsTileRequest

bands = ['B01','B02','B03','B04','B07','B08','B8A', 'B10','B11','B12']
metafiles = ['tileInfo', 'preview', 'qi/MSK_CLOUDS_B00']
data_folder = './AwsData'

request = AwsTileRequest(
    tile=tile_name,
    time=time,
    aws_index=aws_index,
    bands=bands,
    metafiles=metafiles,
    data_folder=data_folder,
    data_collection=DataCollection.SENTINEL2_L1C
)

In [ ]:
request.download_list

In [ ]:
request.config

In [ ]:
# Download sentinel2 data from AWS
request.save_data()

In [ ]:
# !aws s3 cp s3://sentinel-s2-l1c/tiles/10/T/EL/2020/8/15/0/B01.jp2 .

In [ ]:
# Parse the request response 

data_list = request.get_data(redownload=False)

p_b01,p_b02,p_b03,p_b04,p_b07,p_b08,p_b8a,p_b10,p_b11,p_b12,p_tile_info, p_preview, p_cloud_mask = data_list

#### Visualize Raw Data
Along with the spectral bands, Sentinel tiles also include a preview image, let's check that out first to make sure we have the area of interest captured clearly.

In [ ]:
# Preview 

plt.figure(figsize = (12,12))
plt.imshow(p_preview,aspect='auto');

#### Checking Bands
It's also a good practice to spot check a few additional bands to make sure we have everything. Here we plot Band 7 – Vegetation red edge, Band 8 – NIR and Band 8A – Narrow NIR.

In [ ]:
plt.figure(figsize = (36,12));
f, axarr = plt.subplots(1,3,figsize = (24,12));
axarr[0].imshow(p_b07);
axarr[0].title.set_text("Vegetation Red Edge")
axarr[1].imshow(p_b08);
axarr[1].title.set_text("NIR")
axarr[2].imshow(p_b8a);
axarr[2].title.set_text("Narrow NIR")

## Working with Raster Data
Geospatial data is essentially comprised of raster data or vector data. Sentinel-2 uses GeoTIFF, a gridded raster datasets for satellite imagery and terrain models. Rasterio is a Python library that allows to read, inspect, visualize and write geospatial raster data. Here we use `rasterio` to read thee raster arrays and then use this data to create a true color image.

In [ ]:
import rasterio
from rasterio import plot

For getting to the true color images, we will need the blue, green, red and NIR bands.

In [ ]:
# Reeading the required bands with rasterio

band2 = rasterio.open('./AwsData/10TEL,2020-08-15,0/B02.jp2', driver='JP2OpenJPEG') #blue
band3 = rasterio.open('./AwsData/10TEL,2020-08-15,0/B03.jp2', driver='JP2OpenJPEG') #green
band4 = rasterio.open('./AwsData/10TEL,2020-08-15,0/B04.jp2', driver='JP2OpenJPEG') #red
band8 = rasterio.open('./AwsData/10TEL,2020-08-15,0/B08.jp2', driver='JP2OpenJPEG') #nir

#### True Color Image
We can use `rasterio` to create a true color image in `.tiff` format. A true color image has a large file size, please ensure you have at least 2+ GBs of free disk space before exporting.

In [ ]:
#export true color image
trueColor = rasterio.open('./AwsData/lake_shasta.tiff','w',driver='Gtiff',
                         width=band4.width, height=band4.height,
                         count=3,
                         crs=band4.crs,
                         transform=band4.transform,
                         dtype=band4.dtypes[0]
                         )
trueColor.write(band2.read(1),3) #blue
trueColor.write(band3.read(1),2) #green
trueColor.write(band4.read(1),1) #red
trueColor.close()

src = rasterio.open(r"./AwsData/lake_shasta.tiff", count=3)
plot.show(src);

#### Rendering a True Color Image
As you see visualizing a tiff image directly within Jupyter is not very helpful. You will need a GIS software to open and view this. Below in a example that was processed using [QGIS](https://qgis.org/)

In [ ]:
import matplotlib.image as mpimg
plt.figure(figsize = (18,18))
img = mpimg.imread('./images/lake_shasta.png')
imgplot = plt.imshow(img)
plt.show()

## Calculating Spectral Indices
Spectral indices are combinations of the pixel values from two or more spectral bands in a multispectral image. Spectral indices highlight pixels showing the relative abundance or lack of a land-cover type of interest in an image. Let's looks at a couple 

#### Normalized Difference Vegetation Index - NVDI
The normalized difference vegetation index is a simple graphical indicator that can be used to analyze whether or not the target being observed contains live green vegetation. 

It calculated as `NDVI = (NIR – Red) / (NIR + Red)`

In [ ]:
b4 = rasterio.open('./AwsData/10TEL,2020-08-15,0/B04.jp2')
b8 = rasterio.open('./AwsData/10TEL,2020-08-15,0/B08.jp2')

# read Red(b4) and NIR(b8) as arrays
red = b4.read()
nir = b8.read()
ndvi = (nir.astype(float)-red.astype(float))/(nir.astype(float)+red.astype(float))

The `earthpy` package allows easy plotting of visualization of bands, we use it here to visualize the Normalized Difference Vegetation index around the Lake Shasta region.

In [ ]:
import earthpy.spatial as es
import earthpy.plot as ep

In [ ]:
title = "Normalized Difference Vegetation Index (NDVI)"
ep.plot_bands(ndvi, cmap="RdYlGn", cols=1, title=title, vmin=-1, vmax=1);

You can see areas with vegetation in green, areas with dense vegetation as darker shades of green, water bodes generally have low to no vegetation and as such in a contrasting shade of orange. 

#### Normalized Difference Water Index - NDWI
Normalized Difference Water Index (NDWI) is use for analyzing water bodies. The index uses Green and Near infra-red bands of remote sensing images. The NDWI can enhance water information efficiently in most cases.

NDWI is calculated as `NDWI = (GREEEN – NIR) / (GREEN + NIR)`

In [ ]:
b3 = rasterio.open('./AwsData/10TEL,2020-08-15,0/B03.jp2')
b8 = rasterio.open('./AwsData/10TEL,2020-08-15,0/B08.jp2')
# read Gree(b4) annd NIR(b8) as arrays
green = b3.read()
nir = b8.read()
ndwi = (green.astype(float)-nir.astype(float))/(green.astype(float)+nir.astype(float))

In [ ]:
title = "Normalized Difference Water Index (NDWI)"
ep.plot_bands(ndwi, cmap='YlGnBu', cols=1, title=title, vmin=-1, vmax=1); #cmap='YlGnBu'

The above visualization shows us the values plotted for the Lake Shasta region. You can see the lake area in blue and the areas with no or less water in contrasting shades.

## Clean Up (Optional)
Though we did not create any AWS billable resources as part of this exercise, the geographic and GIS data that we downloaded and the images generated may take up significant storage. Make sure to check any storage utilization and delete the files as needed.